In [1]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('ZISHU_API_KEY')
base_url = "http://43.200.7.56:8008/v1"
chat_model = "glm-4-flash"



In [ ]:
import sys
import os

# 将项目根目录添加到 sys.path
project_path = "../"
if project_path not in sys.path:
    # print(project_path)
    sys.path.append(project_path)



../


In [3]:
from typing import List
from zigent.agents import ABCAgent, BaseAgent
from zigent.llm.agent_llms import LLM
from zigent.commons import TaskPackage
from zigent.actions.BaseAction import BaseAction
from zigent.logging.multi_agent_log import AgentLogger
from duckduckgo_search import DDGS

llm = LLM(api_key=api_key, base_url=base_url, model_name=chat_model)
response = llm.run("你是谁？")
print(response)

我是一个人工智能助手，名叫 ChatGLM，是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。


In [11]:
# 假设 BaseAction 是一个基类，定义了 action_name, action_desc, params_doc 等属性
class BaseAction:
    def __init__(self, action_name, action_desc, params_doc):
        self.action_name = action_name
        self.action_desc = action_desc
        self.params_doc = params_doc


class DuckSearchAction(BaseAction):
    def __init__(self, proxy=None) -> None:
        action_name = "DuckDuckGo_Search"
        action_desc = "Using this action to search online content."
        params_doc = {"query": "the search string. be simple."}
        
        # 初始化 DDGS 并配置代理
        self.ddgs = DDGS(proxy=proxy)
        
        super().__init__(
            action_name=action_name, 
            action_desc=action_desc, 
            params_doc=params_doc,
        )

    def __call__(self, query):
        try:
            # 使用 text 方法进行搜索
            results = self.ddgs.text(query)
            return results
        except Exception as e:
            # 错误处理
            print(f"Search failed: {e}")
            return []


In [26]:
#     # 示例 1: 不使用代理
# search_action = DuckSearchAction()
# results = search_action("Python programming")
# print("Results without proxy:")
# for result in results:
#     print(result)

# 示例 2: 使用代理
proxy = {
    "http": "http://61.79.73.225:80",  # 替换为你的代理地址和端口
    "https": "https://61.79.73.225:80",  # 替换为你的代理地址和端口
}

search_action_with_proxy = DuckSearchAction(proxy=proxy["http"])
results_with_proxy = search_action_with_proxy("Python programming")
print("\nResults with proxy:")
for result in results_with_proxy:
    print(result)

Search failed: https://html.duckduckgo.com/html RuntimeError: error sending request for url (https://html.duckduckgo.com/html): client error (Connect)

Caused by:
    0: client error (Connect)
    1: 远程主机强迫关闭了一个现有的连接。 (os error 10054)

Results with proxy:


In [19]:
search_action = DuckSearchAction()
results = search_action("什么是 agent")
print(results)

Search failed: https://lite.duckduckgo.com/lite/ RuntimeError: error sending request for url (https://lite.duckduckgo.com/lite/): operation timed out

Caused by:
    operation timed out
[]


In [ ]:
class DuckSearchAgent(BaseAgent):
    def __init__(
        self,
        llm: LLM,
        actions: List[BaseAction] = [DuckSearchAction()],
        manager: ABCAgent = None,
        **kwargs
    ):
        name = "duck_search_agent"
        role = "You can answer questions by using duck duck go search content."
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager,
            logger=agent_logger,
        )

In [ ]:
def do_search_agent():
    # 创建代理实例
    search_agent = DuckSearchAgent(llm=llm)

    # 创建任务
    task = "what is the found date of microsoft"
    task_pack = TaskPackage(instruction=task)

    # 执行任务并获取响应
    response = search_agent(task_pack)
    print("response:", response)

if __name__ == "__main__":
    do_search_agent()

In [ ]:
BOCHA_API_KEY = os.getenv('BOCHA_API_KEY')
import requests
import json
# 定义Bocha Web Search工具
def bocha_web_search_tool(query: str, count: int = 8) -> str:
    """
    使用Bocha Web Search API进行联网搜索，返回搜索结果的字符串。
    
    参数:
    - query: 搜索关键词
    - count: 返回的搜索结果数量

    返回:
    - 搜索结果的字符串形式
    """
    url = 'https://api.bochaai.com/v1/web-search'
    headers = {
        'Authorization': f'Bearer {BOCHA_API_KEY}',  # 请替换为你的API密钥
        'Content-Type': 'application/json'
    }
    data = {
        "query": query,
        "freshness": "noLimit", # 搜索的时间范围，例如 "oneDay", "oneWeek", "oneMonth", "oneYear", "noLimit"
        "summary": True, # 是否返回长文本摘要总结
        "count": count
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        # 返回给大模型的格式化的搜索结果文本
        # 可以自己对博查的搜索结果进行自定义处理
        return json.dumps(response.json())
    else:
        raise Exception(f"API请求失败，状态码: {response.status_code}, 错误信息: {response.text}")

In [ ]:
class BochaSearchAction(BaseAction):
    def __init__(self) -> None:
        action_name = "Bocha_Search"
        action_desc = "Using this action to search online content."
        params_doc = {"query": "the search string. be simple."}
        super().__init__(
            action_name=action_name, 
            action_desc=action_desc, 
            params_doc=params_doc,
        )

    def __call__(self, query):
        results = bocha_web_search_tool(query)
        rst = json.loads(results)
        result = ""
        for item in rst["data"]["webPages"]["value"]:
            result += item["summary"]
        return result

In [ ]:
search_action = BochaSearchAction()
results = search_action("上海有哪些私立小学在招聘小学英语老师？")
print(results)

In [ ]:
search_action = BochaSearchAction()
class BochaSearchAgent(BaseAgent):
    def __init__(
        self,
        llm: LLM,
        actions: List[BaseAction] = [search_action],
        manager: ABCAgent = None,
        **kwargs
    ):
        name = "bocha_search_agent"
        role = "You can answer questions by using bocha search content."
        super().__init__(
            name=name,
            role=role,
            llm=llm,
            actions=actions,
            manager=manager,
        )

In [ ]:
def do_search_agent():
    # 创建代理实例
    search_agent = BochaSearchAgent(llm=llm)

    # 创建任务
    task = "what is the found date of microsoft"
    task_pack = TaskPackage(instruction=task)

    # 执行任务并获取响应
    response = search_agent(task_pack)
    print("response:", response)

In [ ]:
do_search_agent()